In [1]:
import sys
from pathlib import Path

# Resolve project root (notebook is in analysis/)
PROJECT_ROOT = Path.cwd().resolve().parents[0]
sys.path.insert(0, str(PROJECT_ROOT))

# Define data paths
MUT_PATH = [
    PROJECT_ROOT / "data" / "raw" / "mutations" / "UV_mutations.bed",
    PROJECT_ROOT / "data" / "raw" / "mutations" / "ICGC_WGS_Feb20_mutations.MELA_SKCM.bed"
]
FAI_PATH = PROJECT_ROOT / "data" / "raw" / "reference" / "GRCh37.fa.fai"
FASTA_PATH = PROJECT_ROOT / "data" / "raw" / "reference" / "GRCh37.fa"
TIMING_BW = PROJECT_ROOT / "data" / "raw" / "timing" / "repliSeq_SknshWaveSignalRep1.bigWig"

DNASE_MAP = {
    "mela": PROJECT_ROOT / "data" / "raw" / "DNase-seq" / "mela_ENCFF285GEW.bigWig",
    "kera": PROJECT_ROOT / "data" / "raw" / "DNase-seq" / "kera_ENCFF597YXQ.bigWig",
    "fibr": PROJECT_ROOT / "data" / "raw" / "DNase-seq" / "fibr_ENCFF355OPU.bigWig",
}

# TUMOUR_WHITELIST = [
#     "MELA",
#     "SKCM",
# ]
TUMOUR_WHITELIST = None


In [ ]:
from scripts.grid_search import run_grid_experiment

# Small, fast smoke test
quick_results_1 = run_grid_experiment(
    mut_path=MUT_PATH,
    fai_path=FAI_PATH,
    fasta_path=FASTA_PATH,
    dnase_bigwigs=DNASE_MAP,
    timing_bigwig=TIMING_BW,
    sample_sizes=[5],
    repeats=1,
    base_seed=123,
    counts_raw_bins=[100_000],
    counts_gauss_bins=[100_000],
    inv_dist_gauss_bins=[100_000],
    exp_decay_bins=[100_000],
    exp_decay_adaptive_bins=[100_000],
    track_strategies=["counts_raw"],
    covariate_sets=[
        ["gc", "cpg"],
    ],
    include_trinuc=False,
    chroms=None,  # all chromosomes in the .fai
    standardise_tracks=True,
    standardise_scope="per_chrom",
    verbose=True,
    out_dir=PROJECT_ROOT / "outputs" / "experiments" / "mut_vs_dnase_quickcheck_v1",
    tumour_whitelist=TUMOUR_WHITELIST,
    counts_gauss_sigma_grid=[1.0],
    inv_dist_gauss_pairs=[(0.5, 1_000_000)],
    counts_gauss_sigma_units="bins",
    inv_dist_gauss_sigma_units="bins",
    save_per_bin=False,
)

quick_results_1.sort_values("best_celltype_linear_resid_value").head(5)
quick_results_1.sort_values("best_celltype_local_score_value").head(5)


In [ ]:
from scripts.grid_search import run_grid_experiment

# Quick-but-slightly-larger search (3 runs)
quick_results_2 = run_grid_experiment(
    mut_path=MUT_PATH,
    fai_path=FAI_PATH,
    fasta_path=FASTA_PATH,
    dnase_bigwigs=DNASE_MAP,
    timing_bigwig=TIMING_BW,
    sample_sizes=[5],  # 1
    repeats=1,  # 1
    base_seed=123,
    counts_raw_bins=[100_000],  # 1,
    counts_gauss_bins=[100_000],  # 1,
    inv_dist_gauss_bins=[100_000],  # 1,
    exp_decay_bins=[100_000],  # 1,
    exp_decay_adaptive_bins=[100_000],  # 1,
    track_strategies=["counts_raw", "counts_gauss", "inv_dist_gauss"],  # 3
    covariate_sets=[
        ["gc", "cpg"],  # 1
    ],
    include_trinuc=False,
    chroms=None,  # all chromosomes in the .fai
    standardise_tracks=True,
    standardise_scope="per_chrom",
    verbose=True,
    out_dir=PROJECT_ROOT / "outputs" / "experiments" / "mut_vs_dnase_quickcheck_v2",
    tumour_whitelist=TUMOUR_WHITELIST,
    counts_gauss_sigma_grid=[1.0],
    inv_dist_gauss_sigma_grid=[0.5],
    inv_dist_gauss_max_distance_bp_grid=[1_000_000],
    counts_gauss_sigma_units="bins",
    inv_dist_gauss_sigma_units="bins",
    save_per_bin=False,
)

quick_results_2.sort_values("best_celltype_linear_resid_value").head(10)
quick_results_2.sort_values("best_celltype_local_score_value").head(10)


In [ ]:
from scripts.grid_search import run_grid_experiment

# Full grid search
full_results_1 = run_grid_experiment(
    mut_path=MUT_PATH,
    fai_path=FAI_PATH,
    fasta_path=FASTA_PATH,
    dnase_bigwigs=DNASE_MAP,
    timing_bigwig=TIMING_BW,
    sample_sizes=[1, 2, 3],
    repeats=1,
    base_seed=123,
    counts_raw_bins=[100_000, 1_000_000],
    counts_gauss_bins=[100_000, 1_000_000],
    inv_dist_gauss_bins=[100_000, 1_000_000],
    exp_decay_bins=[100_000, 1_000_000],
    exp_decay_adaptive_bins=[100_000, 1_000_000],
    track_strategies=["counts_raw", "counts_gauss", "inv_dist_gauss"],
    covariate_sets=[["gc", "cpg"]],
    include_trinuc=False,
    chroms=None,  # all chromosomes in the .fai
    standardise_tracks=True,
    standardise_scope="per_chrom",
    tumour_whitelist=TUMOUR_WHITELIST,

    # counts_gauss hyperparameters
    counts_gauss_sigma_grid=[0.5, 1.0, 2.0],

    # inv_dist_gauss paired hyperparameters (sigma, max_distance_bp)
    inv_dist_gauss_pairs=[
        (0.25, 200_000),
        (0.5, 500_000),
        (1.0, 1_000_000),
    ],
    counts_gauss_sigma_units="bins",
    inv_dist_gauss_sigma_units="bins",
    save_per_bin=False,
    out_dir=PROJECT_ROOT / "outputs" / "experiments" / "mut_vs_dnase_full_1",
)

full_results_1.sort_values("best_celltype_linear_resid_value").head(20)
full_results_1.sort_values("best_celltype_local_score_value").head(20)


In [ ]:
from scripts.grid_search import run_grid_experiment

# Focused inv_dist_gauss sweep: 3 bins x 3 pairs (TOTAL RUNS = 18)
inv_results_1 = run_grid_experiment(
    mut_path=MUT_PATH,
    fai_path=FAI_PATH,
    fasta_path=FASTA_PATH,
    dnase_bigwigs=DNASE_MAP,
    timing_bigwig=TIMING_BW,
    sample_sizes=[1],
    repeats=2,
    base_seed=123,

    counts_raw_bins=[25_000, 50_000, 75_000],
    counts_gauss_bins=[25_000, 50_000, 75_000],
    inv_dist_gauss_bins=[25_000, 50_000, 75_000],
    exp_decay_bins=[25_000, 50_000, 75_000],
    exp_decay_adaptive_bins=[25_000, 50_000, 75_000],

    track_strategies=["inv_dist_gauss"],
    covariate_sets=[["gc", "cpg"]],
    include_trinuc=False,
    chroms=None,
    standardise_tracks=True,
    standardise_scope="per_chrom",
    tumour_whitelist=TUMOUR_WHITELIST,

    # (inv_dist_gauss_sigma_bins, inv_dist_gauss_max_distance_bp)
    inv_dist_gauss_pairs=[
        (0.20, 150_000),  # tighter/local
        (0.25, 200_000),  # your best-ish anchor from earlier
        (0.35, 300_000),  # slightly broader sensitivity check
    ],

    counts_gauss_sigma_units="bins",
    inv_dist_gauss_sigma_units="bins",
    save_per_bin=False,
    out_dir=PROJECT_ROOT / "outputs" / "experiments" / "mut_vs_dnase_inv_sweep_1",
)

inv_results_1.sort_values("best_celltype_linear_resid_value").head(20)
inv_results_1.sort_values("best_celltype_local_score_value").head(20)


In [ ]:
from scripts.grid_search import run_grid_experiment

# Exponential decay and adaptive bandwidth tracks (bp-space kernels)
exp_results_1 = run_grid_experiment(
    mut_path=MUT_PATH,
    fai_path=FAI_PATH,
    fasta_path=FASTA_PATH,
    dnase_bigwigs=DNASE_MAP,
    timing_bigwig=TIMING_BW,
    sample_sizes=[5],
    repeats=1,
    base_seed=123,
    counts_raw_bins=[100_000],
    counts_gauss_bins=[100_000],
    inv_dist_gauss_bins=[100_000],
    exp_decay_bins=[100_000],
    exp_decay_adaptive_bins=[100_000],
    track_strategies=["exp_decay", "exp_decay_adaptive"],
    covariate_sets=[["gc", "cpg"]],
    include_trinuc=False,
    chroms=None,
    standardise_tracks=True,
    standardise_scope="per_chrom",
    tumour_whitelist=TUMOUR_WHITELIST,
    exp_decay_decay_bp_grid=[100_000, 200_000],
    exp_decay_max_distance_bp_grid=[1_000_000],
    exp_decay_adaptive_k_grid=[5, 10],
    exp_decay_adaptive_min_bandwidth_bp_grid=[50_000, 100_000],
    exp_decay_adaptive_max_distance_bp_grid=[1_000_000],
    save_per_bin=False,
    out_dir=PROJECT_ROOT / "outputs" / "experiments" / "mut_vs_dnase_exp_adaptive_1",
)

exp_results_1.sort_values("best_celltype_linear_resid_value").head(10)
exp_results_1.sort_values("best_celltype_local_score_value").head(10)


In [2]:
from scripts.grid_search import run_grid_experiment

# Melanoma downsampling experiment
mela_downsample_results = run_grid_experiment(
    mut_path=MUT_PATH,
    fai_path=FAI_PATH,
    fasta_path=FASTA_PATH,
    dnase_bigwigs=DNASE_MAP,
    timing_bigwig=TIMING_BW,
    sample_sizes=[1],
    repeats=5,
    base_seed=123,
    track_strategies=[
        "counts_raw",
        "counts_gauss",
        # "inv_dist_gauss",
        "exp_decay",
        "exp_decay_adaptive",
    ],
    covariate_sets=[
        ["gc", "cpg"],
    ],
    include_trinuc=False,
    chroms=None,
    standardise_tracks=True,
    standardise_scope="per_chrom",
    tumour_whitelist=None,
    counts_raw_bins=[1_000_000],
    counts_gauss_bins=[1_000_000],
    counts_gauss_sigma_grid=[1.0],
    counts_gauss_sigma_units="bins",
    # inv_dist_gauss_bins=[1_000_000],
    # inv_dist_gauss_sigma_grid=[0.5],
    # inv_dist_gauss_sigma_units="bins",
    # inv_dist_gauss_max_distance_bp_grid=[1_000_000],
    exp_decay_bins=[1_000_000],
    exp_decay_decay_bp_grid=[200_000],
    exp_decay_max_distance_bp_grid=[1_000_000],
    exp_decay_adaptive_bins=[1_000_000],
    exp_decay_adaptive_k_grid=[5],
    exp_decay_adaptive_min_bandwidth_bp_grid=[50_000],
    exp_decay_adaptive_max_distance_bp_grid=[1_000_000],
    downsample_counts=[1_000, 5_000, 10_000, 20_000, 50_000],
    save_per_bin=False,
    out_dir=Path("/home/lem/projects/mut-epi-origin/outputs/experiments/mut_vs_dnase_mela_downsample_grid"),
)

mela_downsample_results.sort_values("best_celltype_local_score_value").head(10)


[19:13:44] INFO Session start
[19:13:44] INFO Inputs
[19:13:44] INFO   mutations_bed:       [PosixPath('/home/lem/projects/mut-epi-origin/data/raw/mutations/UV_mutations.bed'), PosixPath('/home/lem/projects/mut-epi-origin/data/raw/mutations/ICGC_WGS_Feb20_mutations.MELA_SKCM.bed')]
[19:13:44] INFO   fasta:               /home/lem/projects/mut-epi-origin/data/raw/reference/GRCh37.fa
[19:13:44] INFO   fai:                 /home/lem/projects/mut-epi-origin/data/raw/reference/GRCh37.fa.fai
[19:13:44] INFO   dnase_bigwigs:       mela, kera, fibr
[19:13:44] INFO   timing_bigwig:       /home/lem/projects/mut-epi-origin/data/raw/timing/repliSeq_SknshWaveSignalRep1.bigWig
[19:13:44] INFO   tumour_whitelist:    none
[19:13:44] INFO Grid
[19:13:44] INFO   chroms:              24
[19:13:44] INFO   configs:             250
[19:13:44] INFO START Counting mutations per sample (min=50000) ...
[19:14:56] INFO DONE Counting mutations per sample (min=50000) (1m16.5s)
[19:14:56] INFO   eligible_samples_mi

KeyboardInterrupt: 

In [ ]:
from scripts.grid_search import run_grid_experiment

# Melanoma downsampling experiment
mela_downsample_results = run_grid_experiment(
    mut_path=MUT_PATH,
    fai_path=FAI_PATH,
    fasta_path=FASTA_PATH,
    dnase_bigwigs=DNASE_MAP,
    timing_bigwig=TIMING_BW,
    sample_sizes=[1],
    repeats=1,
    base_seed=123,
    track_strategies=[
        "counts_raw",
        "counts_gauss",
        # "inv_dist_gauss",
        "exp_decay",
        "exp_decay_adaptive",
    ],
    covariate_sets=[
        ["gc", "cpg"],
    ],
    include_trinuc=False,
    chroms=None,
    standardise_tracks=True,
    standardise_scope="per_chrom",
    tumour_whitelist=None,
    counts_raw_bins=[1_000_000],
    counts_gauss_bins=[1_000_000],
    counts_gauss_sigma_grid=[1.0],
    counts_gauss_sigma_units="bins",
    # inv_dist_gauss_bins=[1_000_000],
    # inv_dist_gauss_sigma_grid=[0.5],
    # inv_dist_gauss_sigma_units="bins",
    # inv_dist_gauss_max_distance_bp_grid=[1_000_000],
    exp_decay_bins=[1_000_000],
    exp_decay_decay_bp_grid=[200_000],
    exp_decay_max_distance_bp_grid=[1_000_000],
    exp_decay_adaptive_bins=[1_000_000],
    exp_decay_adaptive_k_grid=[5],
    exp_decay_adaptive_min_bandwidth_bp_grid=[50_000],
    exp_decay_adaptive_max_distance_bp_grid=[1_000_000],
    downsample_counts=[1_000, 5_000, 10_000, 20_000, 50_000],
    save_per_bin=False,
    out_dir=Path("/home/lem/projects/mut-epi-origin/outputs/experiments/mut_vs_dnase_mela_downsample_grid"),
)

mela_downsample_results.sort_values("best_celltype_local_score_value").head(10)


## Results diagnostics (local score + margins + downsample accuracy)
This section summarises which metrics separate the correct cell type by a larger margin, and how accuracy changes with mutation burden.


In [ ]:
from pathlib import Path
from scripts.analyse_results import (
    analyse_results,
    describe_metric_margin_summary,
    describe_downsample_accuracy,
)

results_path = Path('outputs/experiments/mela_downsample_grid/results.csv')
out = analyse_results(results_path, mode='auto')
print(describe_metric_margin_summary(out))
print('')
print(describe_downsample_accuracy(out))

out['metric_margin_summary_df']

out['downsample_accuracy_overall_df']

out['downsample_accuracy_track_df']
